In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE       MCSE         ESS    
         lp__  927.972599500  3.141348285 0.04966907752 0.1198816949  686.635780
accept_stat__    0.941250451  0.081636553 0.00129078724 0.0013404693 1000.000000
   stepsize__    0.052681625  0.004219204 0.00006671147 0.0006755289   39.009752
  treedepth__    5.791750000  0.476905438 0.00754053707 0.0159079383  898.745556
 n_leapfrog__   66.846000000 27.391748045 0.43310156458 0.6600163945 1000.000000
  divergent__    0.000000000  0.000000000 0.00000000000 0.0000000000         NaN
     energy__ -921.553263250  4.000824593 0.06325859116 0.1523996358  689.177617
            a    1.105476258  0.711432509 0.01124873565 0.0231105942  947.643941
           bp    0.260995373  0.077158416 0.00121998168 0.0025387492  923.691624
  a_society.1   -0.202676598  0.241358584 0.00381621429 0.0054731023 1000.

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*